In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models, Input, Model
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, BatchNormalization, Activation
import numpy as np

# verify if GPU is available
print(tf.test.is_gpu_available())

# set memory growth to true
physical_devices = tf.config.experimental.list_physical_devices('GPU')
print("Setting memory growth to True for GPU: ", physical_devices[0])
tf.config.experimental.set_memory_growth(physical_devices[0], True)

# dont display much info of tensorflow
tf.get_logger().setLevel('ERROR')


2024-01-03 19:32:30.589973: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-01-03 19:32:30.649744: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-03 19:32:30.649792: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-03 19:32:30.652140: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-03 19:32:30.665247: I tensorflow/core/platform/cpu_feature_guar

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
True
Setting memory growth to True for GPU:  PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')


2024-01-03 19:32:33.375040: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-01-03 19:32:33.540565: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-01-03 19:32:33.540856: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [2]:

# Load data
manta_path = "/home/vm/SSL_Project_1/data/processed/bag_2023-07-04_15-23-48/_manta.npy"
xiris_path = "/home/vm/SSL_Project_1/data/processed/bag_2023-07-04_15-23-48/_xiris.npy"
y_path = "/home/vm/SSL_Project_1/data/processed/bag_2023-07-04_15-23-48/_y.npy"
feats_path = "/home/vm/SSL_Project_1/data/processed/bag_2023-07-04_15-23-48/_feats.npy"

# load numpy arrays and display shapes
manta = np.load(manta_path)
xiris = np.load(xiris_path)
y = np.load(y_path)
print("manta shape: ", manta.shape)
print("xiris shape: ", xiris.shape)
print("y shape: ", y.shape) # laser power and velocity

#feats = np.load(feats_path)
#print("feats shape: ", feats.shape)
y = y[:, 0] # only use laser power
print("y shape: ", y.shape)

# normalize y
y = y / np.max(y)

manta shape:  (9587, 320, 320)
xiris shape:  (9587, 320, 320)
y shape:  (9587, 2)
y shape:  (9587,)


In [3]:
# unique values in y
y_unique = np.unique(y)
print("unique values in y: ", y_unique)

# encode y as integers based on unique values
y_encoded = np.zeros(y.shape)
for i in range(len(y_unique)):
    y_encoded[y == y_unique[i]] = i
print("y encoded: ", y_encoded)

unique values in y:  [0.18181818 0.45454545 0.72727273 1.        ]
y encoded:  [0. 0. 0. ... 3. 3. 3.]


In [4]:
# shuffle manta and xiris with y_encoded
idx = np.arange(len(y))
np.random.shuffle(idx)
manta = manta[idx]
xiris = xiris[idx]
y_encoded = y_encoded[idx]


In [5]:
# get first 80% of data as training data
train_split = int(0.8 * len(y))
manta_train = manta[:train_split]
xiris_train = xiris[:train_split]
y_train = y_encoded[:train_split]


# get last 20% of data as test data
manta_test = manta[train_split:]
xiris_test = xiris[train_split:]
y_test = y_encoded[train_split:]

# reshape manta and xiris
X_train = np.concatenate((manta_train, xiris_train), axis=0)
X_test = np.concatenate((manta_test, xiris_test), axis=0)
print("X_train shape: ", X_train.shape)
print("X_test shape: ", X_test.shape)

# reshape y
y_train = np.concatenate((y_train, y_train), axis=0)
y_test = np.concatenate((y_test, y_test), axis=0)
y_train = y_train.reshape(-1, 1)
y_test = y_test.reshape(-1, 1)
print("y_train shape: ", y_train.shape)
print("y_test shape: ", y_test.shape)

X_train shape:  (15338, 320, 320)
X_test shape:  (3836, 320, 320)
y_train shape:  (15338, 1)
y_test shape:  (3836, 1)


In [6]:
""" # concatenate the two inputs (manta and xiris) along rows
x = np.concatenate((manta, xiris), axis=0)
y = np.concatenate((y_encoded, y_encoded), axis=0)
print("x shape: ", x.shape)
print("y shape: ", y.shape)

# split data into train and test (manta as input and y as output) with shuffle as true
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, shuffle=True, random_state=42)
print("x_train shape: ", x_train.shape)
print("y_train shape: ", y_train.shape)
print("x_test shape: ", x_test.shape)
print("y_test shape: ", y_test.shape)
 """

' # concatenate the two inputs (manta and xiris) along rows\nx = np.concatenate((manta, xiris), axis=0)\ny = np.concatenate((y_encoded, y_encoded), axis=0)\nprint("x shape: ", x.shape)\nprint("y shape: ", y.shape)\n\n# split data into train and test (manta as input and y as output) with shuffle as true\nfrom sklearn.model_selection import train_test_split\n\nx_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, shuffle=True, random_state=42)\nprint("x_train shape: ", x_train.shape)\nprint("y_train shape: ", y_train.shape)\nprint("x_test shape: ", x_test.shape)\nprint("y_test shape: ", y_test.shape)\n '

In [7]:
def create_encoder(input_shape=(320, 320, 1)):
    inputs = Input(shape=input_shape)
    x = Conv2D(16, (3, 3), activation='relu')(inputs)
    x = MaxPooling2D((2, 2))(x)
    x = Conv2D(32, (3, 3), activation='relu')(x)
    x = MaxPooling2D((2, 2))(x)

    x = Flatten()(x)
    model = Model(inputs=inputs, outputs=x)
    return model

# add projection head
def add_projection_head(input_shape, encoder, embedding_dim):
    inputs = Input(shape=input_shape)
    features = encoder(inputs)
    # add dense and dropout
    # add batch normalization
    #features = BatchNormalization()(features)
    #features = Dense(512, activation='relu')(features)
    #features = Dropout(0.5)(features)
    outputs = Dense(embedding_dim, activation='relu')(features)
    model = Model(inputs=inputs, outputs=outputs)
    return model

input_shape = (320, 320, 1)
embedding_dim = 128
batch_size = 64
epochs = 20
learning_rate = 0.0001
temperature = 0.05

encoder = create_encoder()
encoder_with_projection_head = add_projection_head(input_shape, encoder, embedding_dim)
encoder_with_projection_head.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 320, 320, 1)]     0         
                                                                 
 model (Functional)          (None, 194688)            4800      
                                                                 
 dense (Dense)               (None, 128)               24920192  
                                                                 
Total params: 24924992 (95.08 MB)
Trainable params: 24924992 (95.08 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


2024-01-03 19:33:03.585458: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-01-03 19:33:03.585829: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-01-03 19:33:03.585938: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [8]:
import tensorflow_addons as tfa
from tensorflow import keras

class SupervisedContrastiveLoss(keras.losses.Loss):
    def __init__(self, temperature=1, name=None):
        super().__init__(name=name)
        self.temperature = temperature

    def __call__(self, labels, feature_vectors, sample_weight=None):
        # Normalize feature vectors
        feature_vectors_normalized = tf.math.l2_normalize(feature_vectors, axis=1)
        # Compute logits
        logits = tf.divide(
            tf.matmul(
                feature_vectors_normalized, tf.transpose(feature_vectors_normalized)
            ),
            self.temperature,
        )
        return tfa.losses.npairs_loss(tf.squeeze(labels), logits)

/home/vm/laser/lib/python3.10/site-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [9]:
encoder_with_projection_head.compile(
    optimizer=keras.optimizers.Adam(learning_rate),
    loss=SupervisedContrastiveLoss(temperature),
)

encoder_with_projection_head.summary()

history = encoder_with_projection_head.fit(
    x=X_train, y=y_train, batch_size=batch_size, epochs=epochs, validation_split=0.2,
)

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 320, 320, 1)]     0         
                                                                 
 model (Functional)          (None, 194688)            4800      
                                                                 
 dense (Dense)               (None, 128)               24920192  
                                                                 
Total params: 24924992 (95.08 MB)
Trainable params: 24924992 (95.08 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


Epoch 1/20


2024-01-03 19:33:22.341368: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8902
2024-01-03 19:33:22.471749: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-01-03 19:33:22.908245: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-01-03 19:33:25.078098: I external/local_xla/xla/service/service.cc:168] XLA service 0x7f7eb0ea88d0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-01-03 19:33:25.078146: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA RTX A6000, Compute Capability 8.6
2024-01-03 19:33:25.085361: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1704310405.268584   50302 device_compiler.h:186] 

192/192 [==============================] - 18s 63ms/step - loss: 3.5299 - val_loss: 3.3375
Epoch 2/20
192/192 [==============================] - 7s 38ms/step - loss: 3.1024 - val_loss: 3.1168
Epoch 3/20
192/192 [==============================] - 8s 41ms/step - loss: 2.9625 - val_loss: 2.9505
Epoch 4/20
192/192 [==============================] - 7s 38ms/step - loss: 2.9001 - val_loss: 2.9246
Epoch 5/20
192/192 [==============================] - 7s 38ms/step - loss: 2.8702 - val_loss: 2.8984
Epoch 6/20
192/192 [==============================] - 7s 38ms/step - loss: 2.8498 - val_loss: 2.8696
Epoch 7/20
192/192 [==============================] - 7s 38ms/step - loss: 2.8316 - val_loss: 2.8736
Epoch 8/20
192/192 [==============================] - 7s 38ms/step - loss: 2.8318 - val_loss: 2.8810
Epoch 9/20
192/192 [==============================] - 7s 38ms/step - loss: 2.8134 - val_loss: 2.8969
Epoch 10/20
192/192 [==============================] - 7s 38ms/step - loss: 2.8097 - val_loss: 2.8749

In [10]:
dropout_rate = 0.2
hidden_units = 128
num_classes = 4
num_epochs = 20
def create_classifier(encoder, trainable=False):

    for layer in encoder.layers:
        layer.trainable = trainable

    inputs = keras.Input(shape=input_shape)
    features = encoder(inputs)
    #features = layers.Dropout(dropout_rate)(features)
    #features = layers.Dense(hidden_units, activation="relu")(features)
    features = layers.Dropout(dropout_rate)(features)
    outputs = layers.Dense(num_classes, activation="softmax")(features)

    model = keras.Model(inputs=inputs, outputs=outputs, name="cifar10-classifier")
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate),
        loss=keras.losses.SparseCategoricalCrossentropy(),
        metrics=[keras.metrics.SparseCategoricalAccuracy()],
    )
    return model

classifier = create_classifier(encoder, trainable=False)

history = classifier.fit(x=X_train, y=y_train, batch_size=batch_size, epochs=num_epochs)



Epoch 1/20
240/240 [==============================] - 6s 22ms/step - loss: 0.9592 - sparse_categorical_accuracy: 0.6994
Epoch 2/20
240/240 [==============================] - 5s 21ms/step - loss: 0.6188 - sparse_categorical_accuracy: 0.8562
Epoch 3/20
240/240 [==============================] - 5s 21ms/step - loss: 0.4792 - sparse_categorical_accuracy: 0.8901
Epoch 4/20
240/240 [==============================] - 5s 20ms/step - loss: 0.4013 - sparse_categorical_accuracy: 0.9122
Epoch 5/20
240/240 [==============================] - 5s 21ms/step - loss: 0.3498 - sparse_categorical_accuracy: 0.9308
Epoch 6/20
240/240 [==============================] - 5s 21ms/step - loss: 0.3130 - sparse_categorical_accuracy: 0.9430
Epoch 7/20
240/240 [==============================] - 5s 21ms/step - loss: 0.2846 - sparse_categorical_accuracy: 0.9501
Epoch 8/20
240/240 [==============================] - 5s 21ms/step - loss: 0.2620 - sparse_categorical_accuracy: 0.9564
Epoch 9/20
240/240 [====================

In [13]:
# evaluate model with x_test and y_test
loss, acc = classifier.evaluate(x=X_test, y=y_test)
print("loss: ", loss)
print("acc: ", acc)

Evaluating model with x_test and y_test...
120/120 [==============================] - 2s 14ms/step - loss: 0.1364 - sparse_categorical_accuracy: 0.9718
loss:  0.136428564786911
acc:  0.9718456864356995


In [11]:
# test model with only manta images -> manta_test with first alph of y_test
y_test_manta = y_test[:len(y_test)//2]
#manta_test = X_test[:len(X_test)//2]

accuracy = classifier.evaluate(x=manta_test, y=y_test_manta)[1]

60/60 [==============================] - 1s 16ms/step - loss: 0.0651 - sparse_categorical_accuracy: 0.9838


In [12]:


# Assuming you have your images and features preprocessed and stored in numpy arrays
# image1, image2, laser_power, velocity

# Define the base network architecture
def create_base_network():
    model = models.Sequential()
    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.Flatten())
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(32))
    return model

# Create two instances of the base network
base_network = create_base_network()
image1_network = models.Model(inputs=base_network.input, outputs=base_network.output)
image2_network = models.Model(inputs=base_network.input, outputs=base_network.output)

# Define the distance layer
class DistanceLayer(layers.Layer):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)

    def call(self, image1, image2, laser_power1, velocity1, laser_power2, velocity2):
        image_distance = tf.reduce_sum(tf.square(image1 - image2), axis=-1)
        laser_power_distance = tf.square(laser_power1 - laser_power2)
        velocity_distance = tf.square(velocity1 - velocity2)
        return image_distance + laser_power_distance + velocity_distance
    

class ContrastiveLossLaserPower(tf.keras.losses.Loss):
    def __init__(self):
        super().__init__()

    def call(self, y_true, y_pred, laser_power1, laser_power2):
        # Calculate the margin as a function of laser power
        margin_laser_power = tf.abs(laser_power1 - laser_power2)

        square_pred = tf.square(y_pred)
        margin_square_laser_power = tf.square(tf.maximum(margin_laser_power - y_pred, 0))
        return tf.reduce_mean(y_true * square_pred + (1 - y_true) * margin_square_laser_power)

class ContrastiveLossVelocity(tf.keras.losses.Loss):
    def __init__(self):
        super().__init__()

    def call(self, y_true, y_pred, velocity1, velocity2):
        # Calculate the margin as a function of velocity
        margin_velocity = tf.abs(velocity1 - velocity2)

        square_pred = tf.square(y_pred)
        margin_square_velocity = tf.square(tf.maximum(margin_velocity - y_pred, 0))
        return tf.reduce_mean(y_true * square_pred + (1 - y_true) * margin_square_velocity)
    
class ContrastiveLoss(tf.keras.losses.Loss):
    def __init__(self):
        super().__init__()

    def call(self, y_true, y_pred, laser_power1, velocity1, laser_power2, velocity2):
        # Calculate the margins as functions of laser power and velocity
        margin_laser_power = tf.abs(laser_power1 - laser_power2)
        margin_velocity = tf.abs(velocity1 - velocity2)

        square_pred = tf.square(y_pred)
        margin_square_laser_power = tf.square(tf.maximum(margin_laser_power - y_pred, 0))
        margin_square_velocity = tf.square(tf.maximum(margin_velocity - y_pred, 0))
        return tf.reduce_mean(y_true * square_pred + (1 - y_true) * (margin_square_laser_power + margin_square_velocity))




# Create the distance layer
distance = distance_layer(image1_network.output, image2_network.output, laser_power1, velocity1, laser_power2, velocity2)
model = models.Model(inputs=[image1_network.input, image2_network.input, laser_power1, velocity1, laser_power2, velocity2], outputs=distance)

# Compile the model
model.compile(optimizer='adam', loss='contrastive_loss')  # You need to define 'contrastive_loss'

# Train the model
# model.fit([image1, image2, laser_power, velocity], labels, epochs=10)


NameError: name 'distance_layer' is not defined

In [ ]:
class CombinedLoss(tf.keras.losses.Loss):
    def __init__(self, alpha=0.5):
        super().__init__()
        self.alpha = alpha
        self.contrastive_loss_laser_power = ContrastiveLossLaserPower()
        self.contrastive_loss_velocity = ContrastiveLossVelocity()

    def call(self, y_true, y_pred, laser_power1, laser_power2, velocity1, velocity2):
        loss_laser_power = self.contrastive_loss_laser_power(y_true, y_pred, laser_power1, laser_power2)
        loss_velocity = self.contrastive_loss_velocity(y_true, y_pred, velocity1, velocity2)
        return self.alpha * loss_laser_power + (1 - self.alpha) * loss_velocity


In [ ]:
class ContrastiveLossLaserPower(tf.keras.losses.Loss):
    def __init__(self):
        super().__init__()

    def call(self, y_true, y_pred, laser_power1, laser_power2):
        # Calculate the margin as a function of laser power
        margin_laser_power = tf.abs(laser_power1 - laser_power2)

        square_pred = tf.square(y_pred)
        margin_square_laser_power = tf.square(tf.maximum(margin_laser_power - y_pred, 0))
        return tf.reduce_mean(y_true * square_pred + (1 - y_true) * margin_square_laser_power)

class ContrastiveLossVelocity(tf.keras.losses.Loss):
    def __init__(self):
        super().__init__()

    def call(self, y_true, y_pred, velocity1, velocity2):
        # Calculate the margin as a function of velocity
        margin_velocity = tf.abs(velocity1 - velocity2)

        square_pred = tf.square(y_pred)
        margin_square_velocity = tf.square(tf.maximum(margin_velocity - y_pred, 0))
        return tf.reduce_mean(y_true * square_pred + (1 - y_true) * margin_square_velocity)
    
model.compile(optimizer='adam', loss=[ContrastiveLossLaserPower(), ContrastiveLossVelocity()])

NameError: name 'model' is not defined